In [12]:
import pandas as pd
df = pd.read_csv('heart_disease_uci.csv')

In [13]:
df.head()

,id,age,sex,dataset,cp,trestbps,chol,fbs,restecg,thalch,exang,oldpeak,slope,ca,thal,num
0,1,63,Male,Cleveland,typical angina,145.0,233.0,True,lv hypertrophy,150.0,False,2.3,downsloping,0.0,fixed defect,0
1,2,67,Male,Cleveland,asymptomatic,160.0,286.0,False,lv hypertrophy,108.0,True,1.5,flat,3.0,normal,2
2,3,67,Male,Cleveland,asymptomatic,120.0,229.0,False,lv hypertrophy,129.0,True,2.6,flat,2.0,reversable defect,1
3,4,37,Male,Cleveland,non-anginal,130.0,250.0,False,normal,187.0,False,3.5,downsloping,0.0,normal,0
4,5,41,Female,Cleveland,atypical angina,130.0,204.0,False,lv hypertrophy,172.0,False,1.4,upsloping,0.0,normal,0


In [14]:
#numerical cols - age, trestbps, chol, thalch, oldpeak, ca
#categorical cols - sex, dataset, cp, fbs, restecg, exang, slope, thal


df = df.drop(['id', 'dataset'], axis=1)

In [15]:
#fix target
df['num'] = df['num'].apply(lambda x:0 if x == 0 else 1)

In [16]:
#encode categorical convert male female to nums
df = pd.get_dummies(df, drop_first=True)

In [17]:
df.head()

,age,trestbps,chol,thalch,oldpeak,ca,num,sex_Male,cp_atypical angina,cp_non-anginal,cp_typical angina,fbs_True,restecg_normal,restecg_st-t abnormality,exang_True,slope_flat,slope_upsloping,thal_normal,thal_reversable defect
0,63,145.0,233.0,150.0,2.3,0.0,0,True,False,False,True,True,False,False,False,False,False,False,False
1,67,160.0,286.0,108.0,1.5,3.0,1,True,False,False,False,False,False,False,True,True,False,True,False
2,67,120.0,229.0,129.0,2.6,2.0,1,True,False,False,False,False,False,False,True,True,False,False,True
3,37,130.0,250.0,187.0,3.5,0.0,0,True,False,True,False,False,True,False,False,False,False,True,False
4,41,130.0,204.0,172.0,1.4,0.0,0,False,True,False,False,False,False,False,False,False,True,True,False


In [19]:
#check missing values
df.isnull().sum()

,0
age,0
trestbps,59
chol,30
thalch,55
oldpeak,62
ca,611
num,0
sex_Male,0
cp_atypical angina,0
cp_non-anginal,0


In [20]:
df = df.fillna(df.median)

In [21]:
df.isnull().sum()

,0
age,0
trestbps,0
chol,0
thalch,0
oldpeak,0
ca,0
num,0
sex_Male,0
cp_atypical angina,0
cp_non-anginal,0


In [22]:
#split x and y
X = df.drop('num', axis=1)
y = df['num']

In [23]:
#train test split
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

In [26]:
X_train.dtypes

,0
age,int64
trestbps,object
chol,object
thalch,object
oldpeak,object
ca,object
sex_Male,bool
cp_atypical angina,bool
cp_non-anginal,bool
cp_typical angina,bool


In [29]:
num_cols = ['trestbps', 'chol', 'thalch', 'oldpeak', 'ca']

for col in num_cols:
  df[col] = pd.to_numeric(df[col], errors='coerce')

df[num_cols] = df[num_cols].fillna(df[num_cols].median())
df = df.replace({True: 1, False: 0})
X = df.drop('num', axis=1)
y = df['num']
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)


/tmp/ipython-input-2578895263.py:7: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df = df.replace({True: 1, False: 0})


In [30]:
#feature scaling since we using logistic reg
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)


In [31]:
pd.DataFrame(X_train).dtypes


,0
0,float64
1,float64
2,float64
3,float64
4,float64
5,float64
6,float64
7,float64
8,float64
9,float64


In [32]:
from sklearn.linear_model import LogisticRegression

log_model = LogisticRegression(max_iter=1000)
log_model.fit(X_train, y_train)

LogisticRegression(max_iter=1000)

In [33]:
y_pred = log_model.predict(X_test)


In [34]:
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

print("Accuracy:", accuracy_score(y_test, y_pred))
print("\nClassification Report:\n", classification_report(y_test, y_pred))
print("\nConfusion Matrix:\n", confusion_matrix(y_test, y_pred))


Accuracy: 0.8043478260869565

Classification Report:
               precision    recall  f1-score   support

           0       0.73      0.81      0.77        75
           1       0.86      0.80      0.83       109

    accuracy                           0.80       184
   macro avg       0.80      0.81      0.80       184
weighted avg       0.81      0.80      0.81       184


Confusion Matrix:
 [[61 14]
 [22 87]]
